In [39]:
using RigidBodyTreeInspector
using RigidBodyDynamics
using PyLCM
using BotCoreLCMTypes

In [40]:
import Flash

In [41]:
urdf = "$(ENV["HOME"])/locomotion/explorations/point-cloud-signed-distance/examples/data/IRB140/urdf/irb_140_convhull.urdf"
package_path = ["$(ENV["HOME"])/locomotion/drake-distro/drake/examples"]
model = Flash.Models.load_urdf(urdf; package_path=package_path)

Manipulator with 8 links and 7 surfaces

In [42]:
squishable = Flash.Models.squishable()

Manipulator with 2 links and 1 surfaces

In [43]:
Flash.Models.merge!(model, squishable)

Manipulator with 9 links and 8 surfaces

In [47]:
state = Flash.ManipulatorState(model);
state.mechanism_state.q[end-2:end] = [0.5, 0, 0.25]
setdirty!(state.mechanism_state)
skin = Flash.skin(state);
Visualizer(DrakeVisualizer.contour_mesh(skin, [-.5, -.5, -.25], [1., .5, 1], 0.01, 0.05))

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 3544xFixedSizeArrays.Point{3,Float64},     faces: 7080xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")]),1,PyLCM.LCM(PyObject <LCM object at 0x320c6b780>))

In [29]:
mechanism = RigidBodyDynamics.parse_urdf(Float64, urdf)

Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: link_1, Edge: joint1
      Vertex: link_2, Edge: joint2
        Vertex: link_3, Edge: joint3
          Vertex: link_4, Edge: joint4
            Vertex: link_5, Edge: joint5
              Vertex: link_6, Edge: joint6

In [30]:
change_joint_type!(mechanism, mechanism.toposortedTree[2].edgeToParentData, QuaternionFloating{Float64}())

Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: link_1, Edge: joint1
      Vertex: link_2, Edge: joint2
        Vertex: link_3, Edge: joint3
          Vertex: link_4, Edge: joint4
            Vertex: link_5, Edge: joint5
              Vertex: link_6, Edge: joint6

In [ ]:
function table()
    geometries = OrderedDict{RigidBody{Float64}, BodyGeometry{Float64}}()

    mechanism = Mechanism(RigidBody{Float64}("world"))
    parent = root_body(mechanism)
    joint = Joint("joint1", QuaternionFloating{Float64}())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    width = 0.5
    thickness = 0.05
    surface_points = Vector{Point3D{SVector{3, Float64}}}()
    for z in [-thickness, thickness]
        for x in [-width, width]
            for y in [-width, width]
                push!(surface_points, )

In [12]:
lc = LCM()

PyLCM.LCM(PyObject <LCM object at 0x320d777b0>)

In [ ]:
log = PyLCM.pylcm[:EventLog]("../squishable/squishable_squish_out.lcm");

In [38]:
for event in log
    if event[:channel] == "KINECT_POINTS_REDUCED"
        @show event
        msg = bot_core[:pointcloud_t][:decode](event["data"])
        publish(lc, "DRAKE_POINTCLOUD_kinect", msg)
        break
    end
end



event = PyObject <lcm.Event object at 0x3240ab5d0>
